In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import re
from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from typing import Sequence
import os
# 设置可见的 GPU 设备为 cuda:0
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

class ChatModel:
    """
    聊天模型类
    model_path: 模型路径 例如：/ssd/xiaxinyuan/code/CS3602_NLP_Final_Project/output/peft_3b/checkpoint-30000
    max_position_embeddings: 最大位置嵌入长度
    database_path: 数据库路径 例如：/ssd/xiaxinyuan/code/CS3602_NLP_Final_Project/chatbot/dataset/hlm.txt
                    若为None，则不使用知识库
    use_how_many_docs: 使用多少个文档
    vectorizer_path: 向量器路径 例如：/ssd/xiaxinyuan/code/CS3602_NLP_Final_Project/chatbot/baai_models/bge-large-zh-v1.5

    主要函数
    model.chat() 聊天函数

    todo: 流式回复；加载已经创建的向量库；系统级prompt设置；长时记忆知识库
    """
    def __init__(self, model_path, torch_dtype="bfloat16", trust_remote_code=True, device_map="auto", use_cache=False,
                 max_position_embeddings=2048,database_path=None,use_how_many_docs=5,vectorstore_path=None,vectorizer_path=None):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(model_path, device_map="auto")
        self.model = self.load_single_model(model_path, torch_dtype, trust_remote_code, device_map, use_cache)
        self.model = self.model.to(self.device)
        self.conversation_history = []
        self.max_position_embeddings=max_position_embeddings
        self.text_splitter= RecursiveCharacterTextSplitter(
                                                            chunk_size=200, # 指定每个文本块的目标大小，这里设置为200个字符。
                                                            chunk_overlap=50, # 指定文本块之间的重叠字符数，这里设置为50个字符。
                                                            length_function=len, # 用于测量文本长度的函数，这里使用Python内置的`len`函数。
                                                            is_separator_regex=False, # 指定`separators`中的分隔符是否应被视为正则表达式，这里设置为False，表示分隔符是字面字符。
                                                            separators=["\n\n",  "\n",   " ",    ".",    ",",     "，",  "。", ] # 定义用于分割文本的分隔符列表。
                                                        )
        self.load_database(database_path,use_how_many_docs)
        self.vectorstore_path=vectorstore_path
        self.vectorizer_path=vectorizer_path
    
    def load_single_model(self, model_path, torch_dtype, trust_remote_code, device_map, use_cache):
        return AutoModelForCausalLM.from_pretrained(
            pretrained_model_name_or_path=model_path, 
            torch_dtype=torch_dtype,
            trust_remote_code=trust_remote_code,
            device_map=device_map,
            use_cache=use_cache
        )
    
    def load_database(self,database_path,use_how_many_docs=5):
        if database_path is None:
            self.vectorstore=None
            return
        if database_path.endswith(".txt"):
            loader=TextLoader(database_path,encoding="utf-8")
            pages=loader.load()
        elif database_path.endswith(".pdf"):
            loader=PyPDFLoader(database_path)
            pages = loader.load_and_split()
        texts=self.text_splitter.split_documents(pages)
        model_name = self.vectorizer_path if self.vectorizer_path is not None else "BAAI/bge-large-zh"
        model_kwargs = {'device': self.device}
        encode_kwargs = {'normalize_embeddings': True} 
        hf=HuggingFaceBgeEmbeddings(model_name=model_name,model_kwargs=model_kwargs,encode_kwargs=encode_kwargs)
        if self.vectorstore_path is not None:
            self.vectorstore=Chroma.from_documents(documents=texts,embedding=hf,persist_directory=self.vectorstore_path)
        else:
            self.vectorstore=Chroma.from_documents(documents=texts,embedding=hf)
        self.retriever=self.vectorstore.as_retriever(search_kwargs={"k": use_how_many_docs})
        
    def format_docs(self,docs: Sequence[Document]) -> str:
        formatted_docs = []
        for i, doc in enumerate(docs):
            doc_string = f"<doc id='{i}'>{doc.page_content}</doc>"
            formatted_docs.insert(0, doc_string)  # 将文档添加到列表的开头
        return "\n".join(formatted_docs)

    def generate_response(self, user_input):
        print(user_input)

        last_round_content=self.conversation_history[-1]["content"]
        match = re.search(r'\[Round (\d+)\]', last_round_content)
        if match:
            last_round = int(match.group(1))
        else:
            last_round = 0

        # 如果存在知识库，则进行相似度搜索，并将其添加到对话历史中
        if self.vectorstore is not None:
            docs=self.retriever.invoke(user_input)
            docs_str=self.format_docs(docs)
            print("docs in knowledge base:",docs_str)
            self.conversation_history.append({"role": "knowledge base", "content": f"[Round {last_round+1}]: {docs_str}"})

        self.conversation_history.append({"role": "user", "content": f"[Round {last_round+1}]: {user_input}"})
        text=self.tokenizer.apply_chat_template(self.conversation_history,tokenize=False,add_generation_prompt=True)
        inputs=self.tokenizer([text],return_tensors="pt").to(self.device)

        # 如果输入的文本长度超过了最大位置嵌入长度，则删除前面的对话历史，直到文本长度小于最大位置嵌入长度
        while len(inputs["input_ids"][0])>self.max_position_embeddings:
            self.conversation_history.pop(1) 
            text=self.tokenizer.apply_chat_template(self.conversation_history,tokenize=False,add_generation_prompt=True)
            inputs=self.tokenizer([text],return_tensors="pt").to(self.device)

        outputs = self.model.generate(**inputs,pad_token_id=self.tokenizer.eos_token_id,max_new_tokens=100)
        response = self.tokenizer.decode(outputs[:, inputs['input_ids'].shape[-1]:][0], skip_special_tokens=True)

        self.conversation_history.append({"role": "assistant", "content": f"[Round {last_round+1}]: {response.strip()}"})
        print(response.strip())
        return
    
    def chat(self):
        while True:
            user_input = input("User: ").strip()
            if user_input.lower() == "\quit":
                print("Session ended. Bye!")
                self.conversation_history=[{"role": "system", "content": "You are a helpful assistant."}]
                break
            elif user_input.lower() == "\\newsession":
                self.conversation_history = [{"role": "system", "content": "You are a helpful assistant."}]
                print("Conversation history cleaned.")
            else:
                self.conversation_history = [{"role": "system", "content": "You are a helpful assistant."}]
                self.generate_response(user_input)
        return
    
# 实例化模型
model_path = "/ssd/xiaxinyuan/code/CS3602_NLP_Final_Project/output/peft_3b/checkpoint-30000"
database_path = "/ssd/xiaxinyuan/code/CS3602_NLP_Final_Project/chatbot/dataset/hlm.txt"
vectorstore_path = "/ssd/xiaxinyuan/code/CS3602_NLP_Final_Project/chatbot/chroma_db_en"
vectorizer_path = "/ssd/xiaxinyuan/code/CS3602_NLP_Final_Project/chatbot/baai_models/bge-large-zh-v1.5"
chat_model = ChatModel(model_path,database_path=database_path,vectorstore_path=vectorstore_path,vectorizer_path=vectorizer_path)

/home/xiaxinyuan/.conda/envs/dino/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.01s/it]
/tmp/ipykernel_3968709/2200532219.py:56: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf=HuggingFaceBgeEmbeddings(model_name=model_name,model_kwargs=model_kwargs,encode_kwargs=encode_kwargs)


In [2]:
chat_model.chat()

贾宝玉是谁
docs in knowledge base: <doc id='4'>."宝玉笑道：“是了，是了，我怎么就忘了。”因问他母亲好，这会子什么勾当.贾芸指贾琏道：“找二叔说句话。”宝玉笑道：“你倒比先越发出挑了，倒象我的儿子。”贾琏笑道：“好不害臊！人家比你大四五岁呢，就替你作儿子了？"宝玉笑道：“你今年十几岁了？"贾芸道：“十八岁。”</doc>
<doc id='3'>."宝玉笑道：“是了，是了，我怎么就忘了。”因问他母亲好，这会子什么勾当.贾芸指贾琏道：“找二叔说句话。”宝玉笑道：“你倒比先越发出挑了，倒象我的儿子。”贾琏笑道：“好不害臊！人家比你大四五岁呢，就替你作儿子了？"宝玉笑道：“你今年十几岁了？"贾芸道：“十八岁。”</doc>
<doc id='2'>."宝玉笑道：“是了，是了，我怎么就忘了。”因问他母亲好，这会子什么勾当.贾芸指贾琏道：“找二叔说句话。”宝玉笑道：“你倒比先越发出挑了，倒象我的儿子。”贾琏笑道：“好不害臊！人家比你大四五岁呢，就替你作儿子了？"宝玉笑道：“你今年十几岁了？"贾芸道：“十八岁。”</doc>
<doc id='1'>."宝玉笑道：“是了，是了，我怎么就忘了。”因问他母亲好，这会子什么勾当.贾芸指贾琏道：“找二叔说句话。”宝玉笑道：“你倒比先越发出挑了，倒象我的儿子。”贾琏笑道：“好不害臊！人家比你大四五岁呢，就替你作儿子了？"宝玉笑道：“你今年十几岁了？"贾芸道：“十八岁。”</doc>
<doc id='0'>.贾政尚未认清,急忙出船,欲待扶住问他是谁.那人已拜了四拜,站起来打了个问讯.贾政才要还揖,迎面一看,不是别人,却是宝玉.贾政吃一大惊,忙问道："可是宝玉么？"那人只不言语,似喜似悲.贾政又问道："你若是宝玉,如何这样打扮,跑到这里？"宝玉未及回言,只见舡头上来了两人,一僧一道,夹住宝玉说道："俗缘已毕,还不快走."说着,三个人飘然登岸而去.贾政不顾地滑,疾忙来赶.见那三人在前,那里赶得上</doc>
贾宝玉是《红楼梦》中的一个主要人物。他是贾府的公子，是贾母的孙子，也是林黛玉和薛宝钗的表兄。贾宝玉是一个多愁善感、情感丰富的人物，他与林黛玉、薛宝钗之间有着复杂的情感纠葛。贾宝玉的形象在《红楼梦》中被描绘得非常生动，他也是中国文学史上一个非常著名的形象。
Session ended. Bye!
